In [3]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = 'cuda:0'
G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load('autovc.ckpt', map_location="cuda:0")
G.load_state_dict(g_checkpoint['model'])

metadata = pickle.load(open('metadata.pkl', "rb"))

spect_vc = []

for sbmt_i in metadata:
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)

In [45]:
metadata[0][2].shape

(90, 80)

In [7]:
[i.shape for k,i in spect_vc]

[(90, 80),
 (90, 80),
 (90, 80),
 (90, 80),
 (89, 80),
 (89, 80),
 (89, 80),
 (89, 80),
 (75, 80),
 (75, 80),
 (75, 80),
 (75, 80),
 (109, 80),
 (109, 80),
 (109, 80),
 (109, 80)]

In [8]:
!ls

3000000-BL.ckpt			  main.py	    README.md
autovc.ckpt			  make_metadata.py  results.pkl
checkpoint_step001000000_ema.pth  make_spect.py     solver_encoder.py
conversion.ipynb		  metadata.pkl	    synthesis.py
data_loader.py			  model_bl.py	    vocoder.ipynb
hparams.py			  model_vc.py	    wavs
LICENSE				  __pycache__


In [9]:
!python make_spect.py

Found directory: ./wavs
p225
p226
p227
p228


In [17]:
!ls $PWD/spmel/p225/*.npy

/home/tony/autovc/spmel/p225/p225_003.npy
/home/tony/autovc/spmel/p225/p225_008.npy
/home/tony/autovc/spmel/p225/p225_011.npy
/home/tony/autovc/spmel/p225/p225_016.npy
/home/tony/autovc/spmel/p225/p225_019.npy
/home/tony/autovc/spmel/p225/p225_020.npy
/home/tony/autovc/spmel/p225/p225_021.npy
/home/tony/autovc/spmel/p225/p225_022.npy
/home/tony/autovc/spmel/p225/p225_023.npy
/home/tony/autovc/spmel/p225/p225_024.npy


In [32]:
!ls $PWD/wavs/p228/*.wav

/home/tony/autovc/wavs/p228/p228_003.wav
/home/tony/autovc/wavs/p228/p228_005.wav
/home/tony/autovc/wavs/p228/p228_008.wav
/home/tony/autovc/wavs/p228/p228_011.wav
/home/tony/autovc/wavs/p228/p228_016.wav
/home/tony/autovc/wavs/p228/p228_019.wav
/home/tony/autovc/wavs/p228/p228_021.wav
/home/tony/autovc/wavs/p228/p228_022.wav
/home/tony/autovc/wavs/p228/p228_023.wav
/home/tony/autovc/wavs/p228/p228_024.wav


In [23]:
a = """/home/tony/autovc/spmel/p225/p225_003.npy
/home/tony/autovc/spmel/p225/p225_008.npy
/home/tony/autovc/spmel/p225/p225_011.npy
/home/tony/autovc/spmel/p225/p225_016.npy
/home/tony/autovc/spmel/p225/p225_019.npy
/home/tony/autovc/spmel/p225/p225_020.npy
/home/tony/autovc/spmel/p225/p225_021.npy
/home/tony/autovc/spmel/p225/p225_022.npy
/home/tony/autovc/spmel/p225/p225_023.npy
/home/tony/autovc/spmel/p225/p225_024.npy"""

In [24]:
[np.load(i).shape for i in a.split("\n")]

[(376, 80),
 (414, 80),
 (369, 80),
 (353, 80),
 (413, 80),
 (328, 80),
 (513, 80),
 (319, 80),
 (589, 80),
 (375, 80)]

In [51]:
!ls $PWD/*.wav

/home/tony/autovc/p225xp225.wav  /home/tony/autovc/p225xp270.wav
/home/tony/autovc/p225xp228.wav  /home/tony/autovc/p228xp225.wav
/home/tony/autovc/p225xp256.wav  /home/tony/autovc/p228xp228.wav


In [27]:
from IPython.display import Audio

In [52]:
Audio(filename='/home/tony/autovc/p225xp225.wav', rate=16000)

In [54]:
Audio('/home/tony/autovc/wavs/p225/p225_003.wav')
Audio("/home/tony/autovc/wavs/p228/p228_011.wav")

In [6]:
! ls ~/D/corpus/data_aishell/wav/test/*.wav

/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0121.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0122.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0123.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0124.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0125.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0126.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0127.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0128.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0129.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0130.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0131.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0132.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0133.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0134.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0135.wav
/home/tony/D/corpus/data_aishell/wav/test/BAC009S0764W0